In [7]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import optuna
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(0)


In [2]:
USE_CUDA = torch.cuda.is_available
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
print(DEVICE)
BATCH_SIZE = 150
EPOCH = 50

cuda


In [8]:
meanR, meanG, meanB = 0.5602434, 0.5241528, 0.5015032
stdR, stdG, stdB = 0.2273327, 0.23729324, 0.24018635

In [4]:
data_transforms = {
    'train': transforms.Compose([transforms.Resize([128, 86]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([meanR, meanG, meanB], [stdR, stdG, stdB])
    ]),
    'test': transforms.Compose([transforms.Resize([128, 86]),
    transforms.ToTensor(),
    transforms.Normalize([meanR, meanG, meanB], [stdR, stdG, stdB])
    ])
}

In [5]:
from collections import Counter
data_dir = './splitted' 
image_datasets = {x: ImageFolder(root=os.path.join(data_dir, x), transform=data_transforms[x]) for x in ['train', 'test']} 

train_dataloader = torch.utils.data.DataLoader(image_datasets['train'], batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(image_datasets['test'], batch_size=BATCH_SIZE, shuffle=True)
class_names = image_datasets['train'].classes


print(dict(Counter(image_datasets['train'].targets)))
print(dict(Counter(image_datasets['test'].targets)))

{0: 3756, 1: 2697, 2: 4249, 3: 3175, 4: 3445, 5: 3395, 6: 4514, 7: 3239, 8: 5108, 9: 3810, 10: 4134, 11: 4074, 12: 4504, 13: 3244, 14: 5107, 15: 3810, 16: 4134, 17: 4074}
{0: 642, 1: 575, 2: 391, 3: 485, 4: 640, 5: 420, 6: 774, 7: 693, 8: 473, 9: 582, 10: 768, 11: 504, 12: 758, 13: 694, 14: 473, 15: 582, 16: 768, 17: 504}


### training and inference routine

## 학습

In [6]:
def train(model, device, train_dataloader, optim, epoch):
    model.train()
    for b_i, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)
        optim.zero_grad()
        pred_prob = model(X)
        loss = F.nll_loss(pred_prob, y) # nll is the negative likelihood loss
        loss.backward()
        optim.step()
    

## 검사

In [8]:
def test(model, device, test_dataloader):
    model.eval()
    loss = 0
    success = 0
    with torch.no_grad():
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            pred_prob = model(X)
            loss += F.nll_loss(pred_prob, y, reduction='sum').item()  # loss summed across the batch
            pred = pred_prob.argmax(dim=1, keepdim=True)  # use argmax to get the most likely prediction
            success += pred.eq(y.view_as(pred)).sum().item()

    loss /= len(test_dataloader.dataset)
    
    accuracy = 100. * success / len(test_dataloader.dataset)

    print('\nTest dataset: Overall Loss: {:.4f}, Overall Accuracy: {}/{} ({:.0f}%)\n'.format(
        loss, success, len(test_dataloader.dataset), accuracy))
    
    return accuracy

### 모델 정의

In [15]:
class ConvNet(nn.Module):
    def __init__(self, trial):
        super(ConvNet, self).__init__()
        num_conv_layers = trial.suggest_int("num_conv_layers", 1, 4)
        num_fc_layers = trial.suggest_int("num_fc_layers", 1, 2)

        self.layers = []
        input_depth = 3  
        for i in range(num_conv_layers):
            output_depth = trial.suggest_int(f"conv_depth_{i}", 16, 64)
            self.layers.append(nn.Conv2d(input_depth, output_depth, 3, 1))
            self.layers.append(nn.ReLU())
            input_depth = output_depth
        self.layers.append(nn.MaxPool2d(2))
        p = trial.suggest_float(f"conv_dropout_{i}", 0.1, 0.4)
        self.layers.append(nn.Dropout(p))
        self.layers.append(nn.Flatten())

        input_feat = self._get_flatten_shape()
        for i in range(num_fc_layers):
            output_feat = trial.suggest_int(f"fc_output_feat_{i}", 16, 64)
            self.layers.append(nn.Linear(input_feat, output_feat))
            self.layers.append(nn.ReLU())
            p = trial.suggest_float(f"fc_dropout_{i}", 0.1, 0.4)
            self.layers.append(nn.Dropout(p))
            input_feat = output_feat
        self.layers.append(nn.Linear(input_feat, 18))
        self.layers.append(nn.LogSoftmax(dim=1))
        
        self.model = nn.Sequential(*self.layers)
    
    def _get_flatten_shape(self):
        conv_model = nn.Sequential(*self.layers)
        op_feat = conv_model(torch.rand(1, 3, 128, 86))
        n_size = op_feat.data.view(1, -1).size(1)
        return n_size
 
    def forward(self, x):
        return self.model(x)

In [18]:
def objective(trial):
    
    model = ConvNet(trial)
    model = model.to(DEVICE)
    opt_name = trial.suggest_categorical("optimizer", ["Adam", "Adadelta", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-1, 5e-1, log=True)
    optimizer = getattr(optim, opt_name)(model.parameters(), lr=lr)
    
    for epoch in range(1, 3):
        train(model, DEVICE, train_dataloader, optimizer, epoch)
        accuracy = test(model, DEVICE, test_dataloader)
        trial.report(accuracy, epoch)
        
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

In [19]:
study = optuna.create_study(study_name="mastering_pytorch", direction="maximize")
study.optimize(objective, n_trials=100, timeout=2000)

pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("results: ")
print("num_trials_conducted: ", len(study.trials))
print("num_trials_pruned: ", len(pruned_trials))
print("num_trials_completed: ", len(complete_trials))

print("results from best trial:")
trial = study.best_trial

print("accuracy: ", trial.value)
print("hyperparameters: ")
for key, value in trial.params.items():
    print("{}: {}".format(key, value))

[I 2022-03-02 09:03:40,328] A new study created in memory with name: mastering_pytorch


epoch: 1 [0/70469 (0%)]	 training loss: 2.896232

Test dataset: Overall Loss: 1.5659, Overall Accuracy: 4848/10726 (45%)

epoch: 2 [0/70469 (0%)]	 training loss: 1.489450


[I 2022-03-02 09:15:34,261] Trial 0 finished with value: 56.91776990490397 and parameters: {'num_conv_layers': 3, 'num_fc_layers': 1, 'conv_depth_0': 35, 'conv_depth_1': 39, 'conv_depth_2': 20, 'conv_dropout_2': 0.287293364348293, 'fc_output_feat_0': 34, 'fc_dropout_0': 0.3207117601152695, 'optimizer': 'Adadelta', 'lr': 0.22323407762068637}. Best is trial 0 with value: 56.91776990490397.



Test dataset: Overall Loss: 1.2601, Overall Accuracy: 6105/10726 (57%)

epoch: 1 [0/70469 (0%)]	 training loss: 2.885364

Test dataset: Overall Loss: 2.8998, Overall Accuracy: 473/10726 (4%)

epoch: 2 [0/70469 (0%)]	 training loss: 2.888412


[I 2022-03-02 09:27:06,517] Trial 1 finished with value: 4.4098452358754425 and parameters: {'num_conv_layers': 4, 'num_fc_layers': 2, 'conv_depth_0': 45, 'conv_depth_1': 42, 'conv_depth_2': 49, 'conv_depth_3': 33, 'conv_dropout_3': 0.2537383564520036, 'fc_output_feat_0': 19, 'fc_dropout_0': 0.15085856736304884, 'fc_output_feat_1': 25, 'fc_dropout_1': 0.2591035083759582, 'optimizer': 'RMSprop', 'lr': 0.1077661825098647}. Best is trial 0 with value: 56.91776990490397.



Test dataset: Overall Loss: 2.9251, Overall Accuracy: 473/10726 (4%)

epoch: 1 [0/70469 (0%)]	 training loss: 2.916198

Test dataset: Overall Loss: nan, Overall Accuracy: 642/10726 (6%)

epoch: 2 [0/70469 (0%)]	 training loss: nan


[I 2022-03-02 09:38:33,605] Trial 2 finished with value: 5.985455901547641 and parameters: {'num_conv_layers': 3, 'num_fc_layers': 1, 'conv_depth_0': 19, 'conv_depth_1': 33, 'conv_depth_2': 60, 'conv_dropout_2': 0.3512702175401138, 'fc_output_feat_0': 22, 'fc_dropout_0': 0.22061991800377206, 'optimizer': 'SGD', 'lr': 0.36930645133838735}. Best is trial 0 with value: 56.91776990490397.



Test dataset: Overall Loss: nan, Overall Accuracy: 642/10726 (6%)

results: 
num_trials_conducted:  3
num_trials_pruned:  0
num_trials_completed:  3
results from best trial:
accuracy:  56.91776990490397
hyperparameters: 
num_conv_layers: 3
num_fc_layers: 1
conv_depth_0: 35
conv_depth_1: 39
conv_depth_2: 20
conv_dropout_2: 0.287293364348293
fc_output_feat_0: 34
fc_dropout_0: 0.3207117601152695
optimizer: Adadelta
lr: 0.22323407762068637


In [9]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        num_conv_layers = 3
        num_fc_layers = 1

        self.layers = []
        input_depth = 3  
        output_depth = 35
        self.layers.append(nn.Conv2d(input_depth, output_depth, 3, 1))
        self.layers.append(nn.ReLU())
        input_depth = output_depth
        output_depth = 39
        self.layers.append(nn.Conv2d(input_depth, output_depth, 3, 1))
        self.layers.append(nn.ReLU())
        input_depth = output_depth
        output_depth = 20
        self.layers.append(nn.Conv2d(input_depth, output_depth, 3, 1))
        self.layers.append(nn.ReLU())
        input_depth = output_depth

        self.layers.append(nn.MaxPool2d(2))
        p = 0.287293364348293
        self.layers.append(nn.Dropout(p))
        self.layers.append(nn.Flatten())

        input_feat = self._get_flatten_shape()
        output_feat = 34
        self.layers.append(nn.Linear(input_feat, output_feat))
        self.layers.append(nn.ReLU())
        p = 0.3207117601152695
        self.layers.append(nn.Dropout(p))
        input_feat = output_feat
        self.layers.append(nn.Linear(input_feat, 18))
        self.layers.append(nn.LogSoftmax(dim=1))
        
        self.model = nn.Sequential(*self.layers)
    
    def _get_flatten_shape(self):
        conv_model = nn.Sequential(*self.layers)
        op_feat = conv_model(torch.rand(1, 3, 128, 86))
        n_size = op_feat.data.view(1, -1).size(1)
        return n_size
 
    def forward(self, x):
        return self.model(x)

In [10]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [11]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss += F.cross_entropy(output, target, reduction = "sum").item()

            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
   
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100.0 * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [12]:
import time
import copy

def train_optuna(model, train_loader, val_loader, optimizer, num_epoches = 30):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(1, num_epoches + 1):
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since

        print('+-+-++-+-++-+-++-+-+ epoch {} +-+-++-+-++-+-++-+-+'.format(epoch))

        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))
        print('val loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    model.load_state_dict(best_model_wts)
    return model



In [13]:
model_base = ConvNet().to(DEVICE)
optimizer = optim.Adadelta(model_base.parameters(), lr = 0.22323407762068637)

In [14]:
optuna = train_optuna(model_base, train_dataloader, test_dataloader, optimizer, num_epoches=EPOCH)
torch.save(optuna, 'optuna.pt')
